In [1]:
from typing import Set
from ray.rllib.env.multi_agent_env import MultiAgentEnv
import random as rnd
import math
from gymnasium.spaces import Discrete, Box, Sequence, Dict
from gymnasium.spaces.utils import flatten, flatten_space
import numpy as np
from IPython.display import clear_output

from ray.rllib.utils.typing import AgentID

class PointCoverageEnv(MultiAgentEnv):

    actions_dict = [(0,-1),(0,1),(1,0),(-1,0),(0,0)]

    def __init__(self, config):
        self.width = config["width"]
        self.height = config["height"]
        self.n_agents = config["n_agents"]
        self.n_targets = config["n_targets"]
        self.max_steps = config["max_steps"] if "max_steps" in config.keys() else None
        self.agents = ['agent-' + str(i) for i in range(self.n_agents)]
        self.observation_space = self.observation_space('agent-0')
        self.action_space = Discrete(5)

    def unflatten_observation_space(self, agent):
       coordinates_space = Box(low=np.array([0, 0]), high=np.array([self.width-1, self.height-1]), dtype=np.int32)
       if self.n_agents > 1:
            return Dict({
                "position": coordinates_space,
                "other_agents": Dict({f"other_agent-{i}": coordinates_space for i in range(self.n_agents-1)}),
                "targets": Dict({f"target-{i}": coordinates_space for i in range(self.n_targets)})
            })
       else:
           return Dict({
                "position": coordinates_space,
                "targets": Dict({f"target-{i}": coordinates_space for i in range(self.n_targets)})
            })

    def observation_space(self, agent):
       return flatten_space(self.unflatten_observation_space(agent))
       #return self.unflatten_observation_space(agent)

    def action_space(self, agent):
        return Discrete(5)
    
    def __get_other_agents(self, agent):
        return [other for other in self.agents if other != agent]

    def __get_random_point(self):
        return (rnd.randint(0, self.width-1), rnd.randint(0, self.height-1))
    
    def __get_observation(self, agent):
        if self.n_agents > 1:
            return flatten(self.unflatten_observation_space(agent), 
                {
                    "position": self.agent_pos[agent],
                    "other_agents": {f"other_agent-{i}": self.agent_pos[other] for i, other in enumerate(self.__get_other_agents(agent))},
                    "targets": {f"target-{i}": pos for i, pos in enumerate(self.targets)}
                }
            )
        else:
            return flatten(self.unflatten_observation_space(agent), 
                {
                    "position": self.agent_pos[agent],
                    "targets": {f"target-{i}": pos for i, pos in enumerate(self.targets)}
                }
            )

    def __get_not_covered_targets(self):
        return set(self.targets) - set(self.agent_pos.values())

    def __is_target_contended(self, target):
        return len([t for t in self.agent_pos.values() if target == t]) > 1

    def __get_reward(self, agent):
        return -1 + self.__get_global_reward()
        if self.agent_pos[agent] in self.targets:
            if self.agent_pos[agent] in [pos[1] for pos in self.old_agent_pos if pos[0] != agent]:
                return -1 # someone was already covering the target -> no +10 reward
            if self.__is_target_contended(self.agent_pos[agent]):
                return -2 # someone arrived at the target at the same time of me -> someone has to leave
            return 10
        else:
            return -1
    
    def __get_global_reward(self):
        return (len(self.not_covered_target) - len(set(self.not_covered_target) - set(self.agent_pos.values())))*10
    
    def __update_agent_position(self, agent, x, y):
        self.agent_pos[agent] = (max(min(self.agent_pos[agent][0] + x, self.width-1), 0),
                                 max(min(self.agent_pos[agent][1] + y, self.height-1), 0))

    def reset(self, seed=None, options=None):
        self.agent_pos = {agent: self.__get_random_point() for agent in self.agents}
        self.targets = [self.__get_random_point() for _ in range(self.n_targets)]
        self.not_covered_target = self.targets.copy()
        self.steps = 0;
        return {agent: self.__get_observation(agent) for agent in self.agents}, {}
     
    def step(self, actions):
        self.steps += 1
        observations, rewards, terminated, truncated, infos = {}, {}, {}, {}, {}

        self.old_agent_pos = self.agent_pos.copy()
        for agent, action in actions.items():
            self.__update_agent_position(agent, self.actions_dict[action][0], self.actions_dict[action][1])

        for agent in actions.keys():
            if not (self.agent_pos[agent] in self.targets and not self.__is_target_contended(self.agent_pos[agent])):
                observations[agent] = self.__get_observation(agent)
                rewards[agent] = self.__get_reward(agent)
                terminated[agent] = False
                truncated[agent] = False
                infos[agent] = {}
        
        if self.max_steps != None and self.steps > self.max_steps:
            truncated['__all__'] = True
        else:
            truncated['__all__'] = False

        self.not_covered_target = list(set(self.not_covered_target) - set(self.agent_pos.values())) 

        terminated['__all__'] = len(self.__get_not_covered_targets()) == 0
        return observations, rewards, terminated, truncated, infos
     
    def render(self, mode='text'):
        str = '_' * (self.width+2) + '\n'
        for i in range(self.height):
            str = str + "|"
            for j in range(self.width):
                if (j,i) in self.agent_pos.values() and (j,i) in self.targets:
                    str = str + '*'
                elif (j,i) in self.agent_pos.values():
                    str = str + 'o'
                elif (j,i) in self.targets:
                    str = str + 'x'
                else:
                    str = str + ' '
            str = str + '|\n'
        str = str + '‾' * (self.width+2)
        print(str)

    def get_agent_ids(self):
       return self.agents

In [14]:
env = PointCoverageEnv({"height": 10, "width": 10, "n_agents": 2, "n_targets": 2})
obs, _ = env.reset() 
print(obs['agent-0'])
print(obs['agent-1'])
env.render()

actions = {"agent-0": 1, "agent-1": 0}
obs, _, _, _, _ = env.step(actions)
print(obs['agent-0'])
env.render()

[7 2 4 7 1 2 6 4]
[4 7 7 2 1 2 6 4]
____________
|          |
|          |
| x     o  |
|          |
|      x   |
|          |
|          |
|    o     |
|          |
|          |
‾‾‾‾‾‾‾‾‾‾‾‾
[7 1 4 8 1 2 6 4]
____________
|          |
|       o  |
| x        |
|          |
|      x   |
|          |
|          |
|          |
|    o     |
|          |
‾‾‾‾‾‾‾‾‾‾‾‾


In [9]:
import ray
ray.shutdown()
ray.init()

def customResultPrint(result):
    print(f"iteration [{result['training_iteration']}] => " +
          f"episode_reward_mean: {result['sampler_results']['episode_reward_mean']}, " +
          f"episode_len_mean: {result['sampler_results']['episode_len_mean']}, " +
          f"agent_steps_trained: {result['info']['num_agent_steps_trained']}, " +
          f"env_steps_trained: {result['info']['num_env_steps_trained']}, " + 
          f"entropy: {result['info']['learner']['default_policy']['learner_stats']['entropy']}, " +
          f"learning_rate: {result['info']['learner']['default_policy']['learner_stats']['cur_lr']}")


2024-05-16 14:54:59,281	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


## Single agent

In [13]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env
from gymnasium.wrappers.time_limit import TimeLimit


register_env("my_env", lambda _: PointCoverageEnv({"height": 10, "width": 10, "n_agents": 1, "n_targets": 1, "max_steps": 100}))

train_batch_size = 2048
sgd_minibatch_size = 256
num_sgd_iter = 10
trainings = 30

total_env_steps = trainings*train_batch_size

print(f"number of different environment steps: {total_env_steps}")


algo = (
    PPOConfig()
    .training(gamma=0.99, 
              lr=0.001,
              kl_coeff=0.5, 
              train_batch_size=train_batch_size, 
              sgd_minibatch_size=sgd_minibatch_size, 
              num_sgd_iter=num_sgd_iter)
    .env_runners(num_env_runners=1)
    .resources(num_gpus=0)
    .environment(env="my_env")
    .build()
)

for i in range(trainings):
    result = algo.train()
    customResultPrint(result)
    if i % 5 == 0:
        checkpoint_dir = algo.save().checkpoint.path
        print(f"Checkpoint saved in directory {checkpoint_dir}")

/home/nicolo/anaconda3/envs/rayEnv/lib/python3.11/site-packages/ray/rllib/algorithms/algorithm.py:521: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
`UnifiedLogger` will be removed in Ray 2.7.
  return UnifiedLogger(config, logdir, loggers=None)
/home/nicolo/anaconda3/envs/rayEnv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API is deprecated and may be removed in future Ray releases. You could suppress this warning by setting env variable PYTHONWARNINGS="ignore::DeprecationWarning"
The `JsonLogger interface is deprecated in favor of the `ray.tune.json.JsonLoggerCallback` interface and will be removed in Ray 2.7.
  self._loggers.append(cls(self.config, self.logdir, self.trial))
/home/nicolo/anaconda3/envs/rayEnv/lib/python3.11/site-packages/ray/tune/logger/unified.py:53: RayDeprecationWarning: This API i

number of different environment steps: 61440


2024-05-16 14:57:55,679	INFO trainable.py:161 -- Trainable.setup took 10.821 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.
2024-05-16 14:57:55,687	WARNING util.py:61 -- Install gputil for GPU system monitoring.


iteration [1] => episode_reward_mean: -87.82608695652173, episode_len_mean: 88.08695652173913, agent_steps_trained: 2048, env_steps_trained: 2048, entropy: 1.5966786593198776, learning_rate: 0.0010000000000000002
Checkpoint saved in directory /tmp/tmp2fua4s14
iteration [2] => episode_reward_mean: -81.38, episode_len_mean: 81.74, agent_steps_trained: 4096, env_steps_trained: 4096, entropy: 1.5709845513105392, learning_rate: 0.0010000000000000002
iteration [3] => episode_reward_mean: -81.34666666666666, episode_len_mean: 81.72, agent_steps_trained: 6144, env_steps_trained: 6144, entropy: 1.5638435751199722, learning_rate: 0.0010000000000000002
iteration [4] => episode_reward_mean: -77.6, episode_len_mean: 78.0, agent_steps_trained: 8192, env_steps_trained: 8192, entropy: 1.5468495473265649, learning_rate: 0.0010000000000000002
iteration [5] => episode_reward_mean: -69.62, episode_len_mean: 70.12, agent_steps_trained: 10240, env_steps_trained: 10240, entropy: 1.5364061310887336, learning_

In [26]:
from IPython.display import clear_output
import time
import torch
from gymnasium.spaces.utils import flatten

env = PointCoverageEnv({"height": 10, "width": 50, "n_agents": 1, "n_targets": 1, "max_steps": 30})
obs_space = env.observation_space
obs, _ = env.reset()
env.render()

for i in range(100):
    actions = algo.compute_actions({agent: o for agent, o in obs.items()})
    print(actions, "\n")
    
    obs, reward, terminated, truncated, info = env.step(actions)
    clear_output()
    print(f"[{i}]")
    env.render()
    print(obs)
    print(reward)
    time.sleep(0.5)

    if terminated['__all__'] or truncated['__all__']:
        break


[29]
____________________________________________________
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                        *                         |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
|                                                  |
‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾‾
{}
{}


## Two Agents

In [12]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env
from gymnasium.wrappers.time_limit import TimeLimit


register_env("my_env", lambda _: PointCoverageEnv({"height": 4, "width": 4, "n_agents": 2, "n_targets": 2, "max_steps": 50}))

train_batch_size = 2048
sgd_minibatch_size = 256
num_sgd_iter = 10
trainings = 50

total_env_steps = trainings*train_batch_size

print(f"number of different environment steps: {total_env_steps}")


algo2 = (
    PPOConfig()
    .training(gamma=0.99, 
              #lr=0.001,
              lr_schedule=[
                [0, 0.01],  
                [1000, 0.001],  
                [10000, 0.001],  
              ],
              kl_coeff=0.5, 
              train_batch_size=train_batch_size, 
              sgd_minibatch_size=sgd_minibatch_size, 
              num_sgd_iter=num_sgd_iter, 
              entropy_coeff_schedule = [
                [0, 1],  # Start with relatively high entropy coefficient
                [20480, 0],  # Gradually decrease entropy coefficient over 10,000 iterations
              ])
    .env_runners(num_env_runners=1)
    .resources(num_gpus=0)
    .environment(env="my_env")
    .build()
)

for i in range(trainings):
    result = algo2.train()
    customResultPrint(result)
    if i % 5 == 0:
        checkpoint_dir = algo2.save().checkpoint.path
        print(f"Checkpoint saved in directory {checkpoint_dir}")

2024-05-16 13:13:09,443	WARNING deprecation.py:50 -- DeprecationWarning: `WorkerSet(num_workers=... OR local_worker=...)` has been deprecated. Use `EnvRunnerGroup(num_env_runners=... AND local_env_runner=...)` instead. This will raise an error in the future!


number of different environment steps: 102400


2024-05-16 13:13:18,534	WARNING util.py:61 -- Install gputil for GPU system monitoring.


iteration [1] => episode_reward_mean: -27.611940298507463, episode_len_mean: 30.402985074626866, agent_steps_trained: 2693, env_steps_trained: 2048, entropy: 1.608395733833313, learning_rate: 0.009999999999999998
Checkpoint saved in directory /tmp/tmprff5mkxb
iteration [2] => episode_reward_mean: -26.11, episode_len_mean: 30.57, agent_steps_trained: 5162, env_steps_trained: 4096, entropy: 1.598888381322225, learning_rate: 0.0010000000000000002
iteration [3] => episode_reward_mean: -25.92, episode_len_mean: 30.53, agent_steps_trained: 7793, env_steps_trained: 6144, entropy: 1.5958628177642822, learning_rate: 0.0010000000000000002
iteration [4] => episode_reward_mean: -21.45, episode_len_mean: 27.31, agent_steps_trained: 10391, env_steps_trained: 8192, entropy: 1.5846111023426055, learning_rate: 0.0010000000000000002
iteration [5] => episode_reward_mean: -18.53, episode_len_mean: 25.63, agent_steps_trained: 12899, env_steps_trained: 10240, entropy: 1.5749792496363322, learning_rate: 0.00

In [15]:
from IPython.display import clear_output
import time

env = PointCoverageEnv({"height": 10, "width": 10, "n_agents": 2, "n_targets": 2})
obs, _ = env.reset()
env.render()

for i in range(100):
    actions = algo2.compute_actions(obs)
    print(actions, "\n")
    obs, reward, terminated, truncated, info = env.step(actions)
    clear_output()
    print(f"[{i}]")
    env.render()
    print(obs)
    print(reward)
    time.sleep(0.5)

    if terminated['__all__'] or truncated['__all__']:
        break


[65]
____________
|   *     o|
|          |
|         x|
|          |
|          |
|          |
|          |
|          |
|          |
|          |
‾‾‾‾‾‾‾‾‾‾‾‾
{'agent-0': array([3, 0, 9, 0, 9, 2, 3, 0], dtype=int32)}
{'agent-0': -1}


KeyboardInterrupt: 

In [26]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.env_checker import check_env

env = PointCoverageEnv({"height": 10, "width": 50, "n_agents": 2, "n_targets": 2})
check_env(env)

model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=25000)

obs, _ = env.reset()
while True:
    action, _states = model.predict(obs)
    obs, rewards, dones, info = env.step(action)

AssertionError: The observation returned by `reset()` method must be a numpy array

## Three Agents

In [16]:
from ray.rllib.algorithms.ppo import PPOConfig
from ray.tune.logger import pretty_print
from ray.tune.registry import register_env
from gymnasium.wrappers.time_limit import TimeLimit


register_env("my_env", lambda _: PointCoverageEnv({"height": 4, "width": 4, "n_agents": 3, "n_targets": 3, "max_steps": 50}))

train_batch_size = 2048
sgd_minibatch_size = 256
num_sgd_iter = 10
trainings = 50

total_env_steps = trainings*train_batch_size

print(f"number of different environment steps: {total_env_steps}")


algo3 = (
    PPOConfig()
    .training(gamma=0.99, 
              #lr=0.001,
              lr_schedule=[
                [0, 0.01],  
                [1000, 0.001],  
                [10000, 0.001],  
              ],
              kl_coeff=0.5, 
              train_batch_size=train_batch_size, 
              sgd_minibatch_size=sgd_minibatch_size, 
              num_sgd_iter=num_sgd_iter, 
              entropy_coeff_schedule = [
                [0, 1],  # Start with relatively high entropy coefficient
                [20480, 0],  # Gradually decrease entropy coefficient over 10,000 iterations
              ])
    .env_runners(num_env_runners=1)
    .resources(num_gpus=0)
    .environment(env="my_env")
    .build()
)

def customResultPrint(result):
    print(f"iteration [{result['training_iteration']}] => " +
          f"episode_reward_mean: {result['sampler_results']['episode_reward_mean']}, " +
          f"episode_len_mean: {result['sampler_results']['episode_len_mean']}, " +
          f"agent_steps_trained: {result['info']['num_agent_steps_trained']}, " +
          f"env_steps_trained: {result['info']['num_env_steps_trained']}, " + 
          f"entropy: {result['info']['learner']['default_policy']['learner_stats']['entropy']}, " +
          f"learning_rate: {result['info']['learner']['default_policy']['learner_stats']['cur_lr']}")

for i in range(trainings):
    result = algo3.train()
    customResultPrint(result)
    if i % 5 == 0:
        checkpoint_dir = algo3.save().checkpoint.path
        print(f"Checkpoint saved in directory {checkpoint_dir}")

2024-05-16 13:22:07,017	WARNING deprecation.py:50 -- DeprecationWarning: `WorkerSet(num_workers=... OR local_worker=...)` has been deprecated. Use `EnvRunnerGroup(num_env_runners=... AND local_env_runner=...)` instead. This will raise an error in the future!


number of different environment steps: 102400


2024-05-16 13:22:13,386	WARNING util.py:61 -- Install gputil for GPU system monitoring.


iteration [1] => episode_reward_mean: -20.545454545454547, episode_len_mean: 36.70909090909091, agent_steps_trained: 3158, env_steps_trained: 2048, entropy: 1.608757315079371, learning_rate: 0.01
Checkpoint saved in directory /tmp/tmpge2rfk8i
iteration [2] => episode_reward_mean: -11.7, episode_len_mean: 32.92, agent_steps_trained: 6138, env_steps_trained: 4096, entropy: 1.599185993454673, learning_rate: 0.0010000000000000005
iteration [3] => episode_reward_mean: -6.57, episode_len_mean: 29.54, agent_steps_trained: 9279, env_steps_trained: 6144, entropy: 1.5917007118463515, learning_rate: 0.0010000000000000005
iteration [4] => episode_reward_mean: -8.28, episode_len_mean: 31.83, agent_steps_trained: 12213, env_steps_trained: 8192, entropy: 1.5871088786558671, learning_rate: 0.0010000000000000005
iteration [5] => episode_reward_mean: -5.63, episode_len_mean: 29.88, agent_steps_trained: 15238, env_steps_trained: 10240, entropy: 1.571779600056735, learning_rate: 0.0010000000000000005
iter

KeyboardInterrupt: 

In [21]:
from IPython.display import clear_output
import time

env = PointCoverageEnv({"height": 10, "width": 10, "n_agents": 3, "n_targets": 3})
obs, _ = env.reset()
env.render()

for i in range(100):
    actions = algo3.compute_actions(obs)
    print(actions, "\n")
    obs, reward, terminated, truncated, info = env.step(actions)
    clear_output()
    print(f"[{i}]")
    env.render()
    print(obs)
    print(reward)
    time.sleep(0.5)

    if terminated['__all__'] or truncated['__all__']:
        break


____________
|          |
|          |
|          |
|       o  |
|          |
|          |
|  o   o   |
|     x    |
|     x    |
|        x |
‾‾‾‾‾‾‾‾‾‾‾‾


NameError: name 'algo3' is not defined